In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import openai

from keymaker.models import chatgpt, gpt4, LlamaCpp
from keymaker.constraints import RegexConstraint, OptionsConstraint, StopsConstraint
from keymaker import Prompt, Completion, CompletionConfig

import json

with open("./examples/config.json") as f:
    openai.api_key = json.loads(f.read())["OPENAI_API_KEY"]

<IPython.core.display.Javascript object>

In [3]:
from typing import Optional


async def print_stream(completion: Optional[Completion]):
    if completion:
        print(repr(completion))


async def yo_stream(completion: Optional[Completion]):
    if completion:
        print("YO " + completion)

<IPython.core.display.Javascript object>

In [4]:
chat_model = chatgpt()

llama_model = LlamaCpp(
    model_path="/Users/nick/Downloads/llama-2-7b-chat.ggmlv3.q3_K_S.bin"
)

llama.cpp: loading model from /Users/nick/Downloads/llama-2-7b-chat.ggmlv3.q3_K_S.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 11 (mostly Q3_K - Small)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: mem required  = 4603.09 MB (+ 1026.00 MB per state)
llama_new_context_with_model: kv self size  =  256.00 MB
AVX = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 0 | NEON = 1 | ARM_FMA = 1 | F16C = 0 | FP16_VA = 1 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 0 | VSX = 0 | 


<IPython.core.display.Javascript object>

In [5]:
from datetime import datetime

user_message = "Hi, my name is Nick."

<IPython.core.display.Javascript object>

In [6]:
my_math_answer = None


def store_my_math(answer):
    global my_math_answer
    my_math_answer = int(answer)
    return "Duh me no know."

<IPython.core.display.Javascript object>

In [7]:
# default model, stream and max_tokens for all completions
prompt = Prompt(
    """Time: {time}
User: {user_msg}
Assistant: Hello, {}{punctuation}
User: Can you write me a poem about a superhero named pandaman?
Assistant:{poem}
User: What is 10+5?
Assistant: The answer is 10+5={math}
""",
    chat_model,
    stream=print_stream,
    max_tokens=25,
)

filled_in = await prompt.format(
    # custom constraint
    CompletionConfig(constraint=OptionsConstraint({"Sam", "Nick"}), stream=yo_stream),
    # can do formatting just like a regular python string
    punctuation="!",
    user_msg=user_message,
    time=datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    poem=CompletionConfig(
        llama_model, max_tokens=250, constraint=StopsConstraint("User|Assistant")
    ),
    math=CompletionConfig(
        llama_model, constraint=RegexConstraint("[0-9]+"), map_fn=store_my_math
    ),
)

YO Nick



llama_print_timings:        load time =  8471.13 ms
llama_print_timings:      sample time =     0.70 ms /     1 runs   (    0.70 ms per token,  1438.85 tokens per second)
llama_print_timings: prompt eval time =  8470.96 ms /    64 tokens (  132.36 ms per token,     7.56 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =  8549.27 ms
Llama.generate: prefix-match hit


Completion(text=' Of', start=6, stop=9, name=poem, chunk=True, score=0.9606816814575414)



llama_print_timings:        load time =  8471.13 ms
llama_print_timings:      sample time =     0.69 ms /     1 runs   (    0.69 ms per token,  1440.92 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   719.54 ms /     1 runs   (  719.54 ms per token,     1.39 tokens per second)
llama_print_timings:       total time =   722.00 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =  8471.13 ms
llama_print_timings:      sample time =     0.71 ms /     1 runs   (    0.71 ms per token,  1416.43 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    81.44 ms /     1 runs   (   81.44 ms per token,    12.28 tokens per second)
llama_print_timings:       total time =    83.75 ms
Llama.generate: prefix-match hit


Completion(text=' course', start=17, stop=24, name=poem, chunk=True, score=0.9998282827367615)
Completion(text=',', start=12, stop=13, name=poem, chunk=True, score=0.7673466883786322)


...

Completion(text='!', start=591, stop=592, name=poem, chunk=True, score=0.20164835957387073)
Completion(text='
', start=592, stop=593, name=poem, chunk=True, score=0.9840566791160391)



llama_print_timings:        load time =  8471.13 ms
llama_print_timings:      sample time =     0.71 ms /     1 runs   (    0.71 ms per token,  1402.52 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    93.34 ms /     1 runs   (   93.34 ms per token,    10.71 tokens per second)
llama_print_timings:       total time =    95.70 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =  8471.13 ms
llama_print_timings:      sample time =     0.70 ms /     1 runs   (    0.70 ms per token,  1430.62 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    79.02 ms /     1 runs   (   79.02 ms per token,    12.65 tokens per second)
llama_print_timings:       total time =    81.35 ms
Llama.generate: prefix-match hit


Completion(text='User', start=599, stop=603, name=poem, chunk=True, score=0.44202450083691575)
Completion(text='1', start=2, stop=3, name=math, chunk=True, score=0.9999913689096965)



llama_print_timings:        load time =  8471.13 ms
llama_print_timings:      sample time =     0.70 ms /     1 runs   (    0.70 ms per token,  1432.66 tokens per second)
llama_print_timings: prompt eval time =  1767.31 ms /    25 tokens (   70.69 ms per token,    14.15 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =  1800.61 ms


<IPython.core.display.Javascript object>

In [8]:
filled_in

Prompt('Time: 2023-07-21 21:48:06
User: Hi, my name is Nick.
Assistant: Hello, Nick!
User: Can you write me a poem about a superhero named pandaman?
Assistant:User
User: What is 10+5?
Assistant: The answer is 10+5=Duh me no know.
')

<IPython.core.display.Javascript object>

In [9]:
filled_in.completions.poem

Completion(text='User', start=151, stop=155, name=poem, chunk=False, score=4.091912637842387e-25)

<IPython.core.display.Javascript object>

In [10]:
filled_in.completions.math

'Duh me no know.'

<IPython.core.display.Javascript object>